In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tqdm.notebook import tqdm
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, LSTM, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
def Skip_AE(seq_length, dim, N_RES=3, N_LAYERS=1, BATCH_SIZE=128):
    tf.keras.backend.clear_session()
    
    en_inputs = []
    shared_latents = []
    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        en_input = Input(shape=[len(selected_t), dim])
        en_inputs.append(en_input)

    de_outputs = []
    for n in range(N_RES):
        for l in range(N_LAYERS):
            X = en_inputs[n] if l == 0 else X
            X = GaussianNoise(0.5)(X) if l == 0 else X
            X = LSTM(64, return_sequences=False if l + 1 == N_LAYERS else True)(X)
        h = Dense(32, activation='relu', kernel_regularizer='l1')(X)
        shared_latents.append(h)

    for n in range(N_RES):
        selected_t = [t for t in range(0, seq_length, 2**n)]
        for l in range(N_LAYERS):
            if l == 0:
                X = Concatenate()(shared_latents)
                X = Dense(128, activation='relu')(X)
                X = Concatenate()([X, shared_latents[n]])
                X = RepeatVector(len(selected_t))(X)
            X = LSTM(64, return_sequences=False if l + 1 == N_LAYERS else True)(X)
        rec_x = Dense(len(selected_t) * dim)(X)
        rec_x = Reshape([len(selected_t), dim])(rec_x)
        de_outputs.append(rec_x)

    model = Model(inputs=en_inputs, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

    return model

### Yahoo S5

In [4]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [5]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.33333329444444565 0.09999998800000123 0.49999993750000654
yahoo_A1 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A1 0.5714285142857172 0.1250000006249989 0.49999993333334053
yahoo_A1 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A1 0.8888888197530895 0.25000000993055255 -0.0
yahoo_A1 0.5714285142857172 0.1999999860000008 -0.0
yahoo_A1 0.9999998500000123 0.49999990000001493 0.9999998750000131
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.33333329444444565 0.0 0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.7499999343750033 0.0 0.0
yahoo_A1 0.7499999343750033 0.1666666744444422 -0.0
yahoo_A1 0.7499999343750033 0.1666666744444422 -0.0
yahoo_A1 0.7499999343750033 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.9999999166666703 0.7499999041666788 0.9999999166666721
yahoo_A1 0.9999999166666703 0.49999996666666835 0.999999916666672

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.4999999500000025 0.0 0.0
yahoo_A3 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A3 0.7999999360000031 5.277777344711406e-09 0.0
yahoo_A3 0.4999999500000025 0.0 0.0
yahoo_A3 0.9090908429752097 0.0 0.0
yahoo_A3 0.4999999500000025 0.10000000133333238 0.0
yahoo_A3 0.7999999360000031 0.2777777723148143 -0.0
yahoo_A3 0.6666666074074105 0.0 0.0
yahoo_A3 0.7999999360000031 0.0 0.0
yahoo_A3 0.9999999333333361 0.49999998333333373 0.0
yahoo_A3 0.9090908429752097 4.083333052458349e-09 0.0
yahoo_A3 0.9090908429752097 0.0 0.0
yahoo_A3 0.28571425306122533 0.08333332361111209 0.0
yahoo_A3 0.7999999360000031 3.5277772680152144e-08 0.0
yahoo_A3 0.28571425306122533 0.0 0.0
yahoo_A3 0.9090908429752097 0.0 0.0
yahoo_A3 0.4999999500000025 0.0 0.0
yahoo_A3 0.4999999500000025 0.0 0.0
yahoo_A3 0.6666666074074105 0.0 0.0
yahoo_A3 0.28571425306122533 0.0 0.0
yahoo_A3 0.28571425306122533 0.0 0.0
yahoo_A3 0.6666666074074105 0.0 0.0
yahoo_A3 0.9090908429752097 0.0 0.0
yahoo_A3 0.28571425306122533 0.0

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.6666666074074105 0.0 0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A4 0.6666666074074105 0.0 0.0
yahoo_A4 0.4999999500000025 0.16666665555555618 -0.0
yahoo_A4 0.9090908429752097 0.2500000140138856 -0.0
yahoo_A4 0.7999999360000031 0.16666667972221952 -0.0
yahoo_A4 0.4999999500000025 0.0 0.0
yahoo_A4 0.4999999500000025 0.0 0.0
yahoo_A4 0.9090908429752097 0.0 0.0
yahoo_A4 0.7999999360000031 5.277777344711406e-09 0.0
yahoo_A4 0.9090908429752097 0.0 0.0
yahoo_A4 0.6666666074074105 0.0 0.0
yahoo_A4 0.6666666074074105 2.374999645729026e-08 0.0
yahoo_A4 0.28571425306122533 0.0 0.0
yahoo_A4 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A4 0.4999999500000025 0.0 0.0
yahoo_A4 0.9090908429752097 0.0 0.0
yahoo_A4 0.7999999360000031 0.0 0.0
yahoo_A4 0.9999999333333361 1.666666540856017e-08 0.0
yahoo_A4 0.9090908429752097 0.0 0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 0.0
yahoo_A4 0.7999999360000031 0.16666667972221952 -0.0
yahoo_A4 0.7999999360000031 0.0 0.0

In [6]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.467616,0.106548,0.127976
yahoo_A2,0.380852,0.064339,0.012798
yahoo_A3,0.701350,0.068992,0.016304
yahoo_A4,0.683027,0.045732,0.003049


### NASA

In [7]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [8]:
for loader in [load_nasa]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.2696078192101617 0.06842696238862755 0.3203802296909952
D2 0.3080684237872245 0.12791586174039554 0.5657177134319693


In [9]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.269608,0.068427,0.320380
D2,0.308068,0.127916,0.565718


### SMD

In [10]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [11]:
for loader in [load_smd]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)
      
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.23153690558205123 0.06546275382269463 0.49999999900806086
smd 0.14105792132682896 0.0 0.0
smd 0.18226599319687592 0.07074495616164417 0.5063779405122146
smd 0.17326731082124444 0.04742547410638876 0.4970059864551308
smd 0.09384163322984918 0.024615384453964495 0.5623229425018206
smd 0.48635231608100876 0.38265005673946323 0.5001841448922145
smd 0.29561198390945814 0.13974924317855564 0.484174067026219
smd 0.34074069530864787 0.14826535733134785 0.5727067561550081
smd 0.21307504139204825 0.11685781012077137 0.4954312968267198
smd 0.3194102924134789 0.15403875202507894 0.5505889208607719
smd 0.12690354134994566 0.03387533860870587 0.49999999785465116
smd 0.29953914490114675 0.0880856216870794 0.1910108099206242
smd 0.2635293887601403 0.07559624965145086 0.4474072238676355
smd 0.08547007725180877 0.022321428454838966 0.4999999973831775
smd 0.21307504139204825 0.05418728927328205 0.32769341206410907
smd 0.026737965262089488 0.006775067613340092 0.4999999898626375
smd 0.19028338351472

In [12]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.189399,0.063338,0.422885


### ECG

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [14]:
for loader in [load_ecg]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.3030302764003694 0.0975019253577718 0.3301630386334772
D2 0.3043477989603044 0.0 0.0
D3 0.11009173251409898 0.0 0.0
D4 0.2999999730000019 0.0 0.0
D5 0.2999999730000019 0.10826376364332546 0.3558558491562026
D6 0.3157894238227215 0.15421233477483748 0.6240399286026933
D7 0.05079364581103594 0.0 0.0
D8 0.21276593798098847 0.0 0.0
D9 0.417910413455115 0.0 0.0


In [15]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.303030,0.097502,0.330163
D2,0.304348,0.000000,0.000000
D3,0.110092,0.000000,0.000000
D4,0.300000,0.000000,0.000000
D5,0.300000,0.108264,0.355856
D6,0.315789,0.154212,0.624040
D7,0.050794,0.000000,0.000000
D8,0.212766,0.000000,0.000000
D9,0.417910,0.000000,0.000000


### Power Demand

In [16]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [17]:
for loader in [load_power_demand]:
    datasets = loader(512, 256)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.7499999343750033 0.1666666744444422 -0.0


In [18]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.75,0.166667,0.0


### 2D Gesture

In [19]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [20]:
for loader in [load_gesture]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        N_RES = 3

        seq_length, dim = X_train.shape[1], X_train.shape[2]
        X_train_reverse = np.flip(X_train, axis=1)
        X_test_reverse = np.flip(X_test, axis=1)

        X_train_by_res = []
        X_test_by_res = []
        X_train_by_res_reverse = []
        
        for n in range(N_RES):
            selected_t = [t for t in range(0, seq_length, 2**n)]
            X_train_by_res.append(X_train[:, selected_t, :])
            X_test_by_res.append(X_test[:, selected_t, :])
            X_train_by_res_reverse.append(X_train_reverse[:, selected_t, :])
        
        model = Skip_AE(seq_length, dim, N_RES, N_LAYERS=2, BATCH_SIZE=128)
        history = model.fit(X_train_by_res, X_train_by_res_reverse, epochs=50, batch_size=128, validation_split=0.3, verbose=0,
                            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)]) 
        
        X_test_rec = [np.flip(rec, axis=1) for rec in model.predict(X_test_by_res)]
        scores = evaluate(X_test, X_test_rec[0], y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.42201831455264965 0.0 0.0


In [21]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.422018,0.0,0.0
